In [3]:
!pip install -q -U kaggle_environments

In [4]:
import itertools 
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
from kaggle_environments import make, evaluate

In [5]:
%%writefile common.py

'''
общий модуль содержащий общие переменные функции и классы
'''

ROCK = 0
'''
Переменная хранящая значение для сущности камень. Значение равно 0
'''

PAPER = 1
'''
Переменная хранящая значение для сущности бумага. Значение равно 1
'''

SCISSORS = 2
'''
Переменная хранящая значение для сущности ножнецы. Значение равно 2
'''

RPS_VALUES = (ROCK, PAPER, SCISSORS)
'''
Все используемые значения в игре
'''

COUNTER_VALUES = {ROCK:PAPER, PAPER:SCISSORS, SCISSORS:ROCK}
'''
Словарь содержащий пары значение/противозначение, где \
противозначение всегда имеет преоритет в сравнении с значением.
Например:
камень/бумага, где бумага побеждает камень
'''

AGENTS = {
    'paper':'paper_agent.py',
    'rock':'rock_agent.py',
    'scissors':'scissors_agent.py',
    'copy_opponent':'copy_opponent_agent.py',
    'reactionary':'reactionary_agent.py',
    'counter_reactionar':'counter_reactionary_agent.py',
    'statistical':'statistical_agent.py',
    'randomizer':'randomizer_agent.py',
    'loop':'loop_agent.py',
    'LinearRegression':'lreg_agent.py',
    'spline_interp':'spline_interp_agent.py',
    'linear_interp':'linear_interp_agent.py',
    'contr_opponent':'contr_opponent_agent.py',
}
'''
словарь содержащий информацию о игровых агентах 
в виде название:файл
'''


Overwriting common.py


In [9]:
import common as cmn

In [11]:
%%writefile contr_opponent_agent.py

import random
import common as cmn

class contr_opponent():
     '''
игровой агент стратегия которого заключается в том чтобы \
возвращать контрзначение для последнего значения выставленного опонентом

    '''
     
     def __call__(self, observation, configuration):        
        # в случае первой итерации, когда нет информации о последнем 
        # значении выставленном оппонентом, выбирается случайное значение 
        # из списка допустимых
        if observation.step == 0:
            return random.randrange(0, configuration.signs)
        # в случае когда есть информация о последнем значении 
        # выставленном оппонентом
        else:
            return cmn.COUNTER_VALUES[observation.lastOpponentAction]

            
agent =  contr_opponent()

# метод вызываемый из kaggle_environments
def call_agent(obs, conf):
    return agent(obs, conf)


Overwriting contr_opponent_agent.py


In [13]:
%%writefile copy_opponent_agent.py

import random

class copy_opponent():
    '''
игровой агент стратегия которого заключается в том чтобы копировать последнее\
значение, выставленное опонентом
    '''

    def __call__(self, observation, configuration):
        # в случае первой итерации, когда нет информации о последнем 
        # значении выставленном оппонентом, выбирается случайное значение 
        # из списка допустимых
        if observation.step == 0:
            return random.randrange(0, configuration.signs)
        # в случае когда есть информация о последнем значении 
        # выставленном оппонентом
        else:
            return observation.lastOpponentAction

agent =  copy_opponent()

# метод вызываемый из kaggle_environments
def call_agent(obs, conf):
    return agent(obs, conf)


Overwriting copy_opponent_agent.py


In [17]:
%%writefile counter_reactionary_agent.py

import random
from  kaggle_environments.envs.rps.utils import get_score

class counter_reactionary():
    '''
игровой агент стратегия которого заключается в том чтобы \
менять следующую значение агента значение, противоположное \
предыдущему значению оппонента
    '''
    
    def __init__(self):
        self.last_counter_action = None
    
    def __call__(self, observation, configuration):
        if observation.step == 0:
            # инициируем случайным образом значение для первого раунда
            self.last_counter_action = random.randrange(0, configuration.signs)
        else:
            # оцениваем результат предыдущего раунда
            score = get_score(self.last_counter_action, observation.lastOpponentAction)
            if score == 1:
                # меняем значение на предыдущий результат оппонента
                self.last_counter_action = (self.last_counter_action + 2) % configuration.signs
            else:
                # меняем значение на противоположное по значимости предыдущему результату оппонента
                self.last_counter_action = (observation.lastOpponentAction + 1) % configuration.signs
        return self.last_counter_action


agent = counter_reactionary()

# метод вызываемый из kaggle_environments
def call_agent(obs, conf):
    return agent(obs, conf)


Overwriting counter_reactionary_agent.py


In [19]:
%%writefile linear_interp_agent.py

import random
import numpy as np
from scipy.interpolate import interp1d
import common as cmn

class linear_interp():
    '''
игровой агент стратегия которого заключается в том чтобы \
рассчитывать при помощи линейной интерполяции следующее значение оппонента \
и возвращать его противозначение
    '''

    def __init__(self):
        self.opponent_values = []

    def __call__(self, observation, configuration):
        if observation.step == 0:
            # в случае начальной итерации
            # инициализируем массив накопления данных
            self.opponent_values = []
            # возвращаем случайное значение
            return random.randrange(0, configuration.signs)
        else:
            # сохраняем последнее значение оппонента в массив
            self.opponent_values.append(observation.lastOpponentAction)
            # генерируем порследовательность входных значений интерполяции
            # по факту это порядковые номера выходных значений
            x_data = list(range(1, len(self.opponent_values) + 1))
            # определяем выходные значения (для наглядности)
            y_data = self.opponent_values
            # формируем приемлемые по типу аргументы для функции интерполяции
            x = np.array(x_data)
            y = np.array(y_data)
            # создаем интерполятор
            interp = interp1d(x, y, kind='linear', fill_value='extrapolate')
            # формируем значение аргумента, для которого нужно получить 
            # прогнозируемое значение
            next_x = max(x_data) + 1
            # формируем приемлемые по типу аргументы для функции интерполяции
            new_x = np.array([next_x])
            # вычисляем прогнозируемое значение стратегии оппонента
            y_pred = interp(new_x)
            predicted_val = y_pred[0] 
            # преобразуем полученное значение в область допустимых значений
            corrected_val = min(cmn.RPS_VALUES , key=lambda x: abs(x - predicted_val))
            # получаем и возвращаем контр-значение для спрогнозированного значения
            counter_val = cmn.COUNTER_VALUES[corrected_val]
            return counter_val

agent = linear_interp()

# метод вызываемый из kaggle_environments
def call_agent(obs, conf):
    return agent(obs, conf)


Overwriting linear_interp_agent.py


In [21]:
%%writefile loop_agent.py

import common as cmn

class loop():
    '''
игровой агент стратегия которого заключается в том \
чтобы возвращать RPS значения по кругу
    '''
    def __init__(self):
        self.loop_idx = None

    def __call__(self, observation, configuration):
        if observation.step == 0:
            # первоначальная инициализация 
            # для первой итерации сравнения агентов
            self.loop_idx = 0
        else:
            # изменяем индекс массива
            self.loop_idx += 1
            if self.loop_idx >= len(cmn.RPS_VALUES):
                self.loop_idx = 0
        # возвращаем значение для текущей итерации
        return cmn.RPS_VALUES[self.loop_idx]

agent = loop()

# метод вызываемый из kaggle_environments
def call_agent(obs, conf):
    return agent(obs, conf)


Overwriting loop_agent.py


In [23]:
%%writefile lreg_agent.py

import random
import numpy as np
from sklearn.linear_model import LinearRegression
import common as cmn

class linear_regression():
    '''
игровой агент стратегия которого заключается в том чтобы \
рассчитывать по линейной регрессии следующее значение оппонента \
и возвращать его противозначение
    '''
    
    def __init__(self):
        self.opponent_values = []

    def __call__(self, observation, configuration):
        if observation.step == 0:
            # в случае начальной итерации
            # инициализируем массив накопления данных
            self.opponent_values = []
            # возвращаем случайное значение
            return random.randrange(0, configuration.signs)
        else:
            # сохраняем последнее значение оппонента в массив
            self.opponent_values.append(observation.lastOpponentAction)
            # генерируем порследовательность входных значений интерполяции
            # по факту это порядковые номера выходных значений
            x_data = list(range(1, len(self.opponent_values) + 1))
            # определяем выходные значения (для наглядности)
            y_data = self.opponent_values
            # формируем приемлемые по типу аргументы для функции интерполяции
            x = np.array(x_data).reshape((-1, 1))
            y = np.array(y_data)
            # создаем модель регрессии и вычисляем оптимальные значение весов
            model = LinearRegression().fit(x, y)
            # вычисляем прогнозируемое значение стратегии оппонента
            y_pred = model.predict(x)
            predicted_val = y_pred[0] 
            # преобразуем полученное значение в область допустимых значений
            corrected_val = \
                min(cmn.RPS_VALUES , key=lambda x: abs(x - predicted_val))
            # получаем и возвращаем контр-значение 
            # для спрогнозированного значения
            counter_val = cmn.COUNTER_VALUES[corrected_val]
            return counter_val

agent = linear_regression()

# метод вызываемый из kaggle_environments
def call_agent(obs, conf):
    return agent(obs, conf)


Overwriting lreg_agent.py


In [25]:
%%writefile paper_agent.py

import common as cmn

class paper():
    '''
игровой агент стратегия которого заключается в том \
чтобы всегда возвращать бумагу
   '''
    
    def __call__(self, observation, configuration):
       # всегда возвращаем бумагу
       return cmn.PAPER

agent = paper()

# метод вызываемый из kaggle_environments
def call_agent(obs, conf):
    return agent(obs, conf)


Overwriting paper_agent.py


In [27]:
%%writefile randomizer_agent.py

import random

class randomizer():
    '''
игровой агент стратегия которого заключается в том \
чтобы всегда возвращать случайное значение
    '''

    def __call__(self, observation, configuration):
        # возвращаем случайное значение
        return random.randrange(0, configuration.signs)

agent = randomizer()

# метод вызываемый из kaggle_environments
def call_agent(obs, conf):
    return agent(obs, conf)


Overwriting randomizer_agent.py


In [24]:
%%writefile reactionary_agent.py

import random
from  kaggle_environments.envs.rps.utils import get_score

class reactionary():
    '''
игровой агент стратегия которого заключается в том чтобы \
в случае проигрыша или ничьей менять значение на противоположное \
по значимости предыдущему результату оппонента
    '''

    def __init__(self):
        self.last_react_action = None

    def __call__(self, observation, configuration):
        if observation.step == 0:
            # инициируем случайным образом значение для первого раунда
            self.last_react_action = random.randrange(0, configuration.signs)
        else:
            # оцениваем результат предыдущего раунда
            score = get_score(self.last_react_action, observation.lastOpponentAction)
            # в случае проигрыша или ничьей 
            if score <= 1:
                # меняем значение на противоположное по значимости 
                # предыдущему результату оппонента
                self.last_react_action = \
                    (observation.lastOpponentAction + 1) % configuration.signs
        return self.last_react_action

agent = reactionary()

# метод вызываемый из kaggle_environments
def call_agent(obs, conf):
    return agent(obs, conf)


Overwriting reactionary_agent.py


In [29]:
%%writefile rock_agent.py

import common as cmn

class rock():
    '''
игровой агент стратегия которого заключается в том \
чтобы всегда возвращать камень
    '''

    def __call__(self, observation, configuration):
        # всегда возвращаем камень
        return cmn.ROCK

agent = rock()

# метод вызываемый из kaggle_environments
def call_agent(obs, conf):
    return agent(obs, conf)


Overwriting rock_agent.py


In [31]:
%%writefile scissors_agent.py

import common as cmn

class scissors():
    '''
игровой агент стратегия которого заключается в том \
чтобы всегда возвращать ножницы
    '''

    def __call__(self, observation, configuration):
      # всегда возвращаем ножницы
      return cmn.SCISSORS

agent = scissors()

# метод вызываемый из kaggle_environments
def call_agent(obs, conf):
    return agent(obs, conf)


Overwriting scissors_agent.py


In [33]:
%%writefile spline_interp_agent.py

import random
import numpy as np
from scipy.interpolate import interp1d
import common as cmn

class spline_interp():
    '''
игровой агент стратегия которого заключается в том чтобы \
рассчитывать при помощи сплайн-интерполяции следующее значение оппонента \
и возвращать его противозначение
    '''

    def __init__(self):
        self.opponent_values = []

    def __call__(self, observation, configuration):
        if observation.step < 4:
            if observation.step == 0:
                # в случае начальной итерации
                # инициализируем массив накопления данных
                self.opponent_values = []
            else:
                # продолжаем накапливать данные 
                # до достижения минимально необходжимого количества
                self.opponent_values.append(observation.lastOpponentAction)
            # возвращаем случайное значение
            return random.randrange(0, configuration.signs)
        else:
            # сохраняем последнее значение оппонента в массив
            self.opponent_values.append(observation.lastOpponentAction)
            # генерируем порследовательность входных значений интерполяции
            # по факту это порядковые номера выходных значений
            x_data = list(range(1, len(self.opponent_values) + 1))
            # определяем выходные значения (для наглядности)
            y_data = self.opponent_values
            # формируем приемлемые по типу аргументы для функции интерполяции
            x = np.array(x_data)
            y = np.array(y_data)
            # создаем интерполятор
            interp = interp1d(x, y, kind='cubic', fill_value='extrapolate') 
            # формируем значение аргумента, для которого нужно получить 
            # прогнозируемое значение
            next_x = max(x_data) + 1
            # формируем приемлемые по типу аргументы для функции интерполяции
            new_x = np.array([next_x])
            # вычисляем прогнозируемое значение стратегии оппонента
            y_pred = interp(new_x)
            predicted_val = y_pred[0] 
            # преобразуем полученное значение в область допустимых значений
            corrected_val = min(cmn.RPS_VALUES , key=lambda x: abs(x - predicted_val))
            # получаем и возвращаем контр-значение для спрогнозированного значения
            counter_val = cmn.COUNTER_VALUES[corrected_val]
            return counter_val

agent = spline_interp()

# метод вызываемый из kaggle_environments
def call_agent(obs, conf):
    return agent(obs, conf)


Overwriting spline_interp_agent.py


In [35]:
%%writefile statistical_agent.py

import random

class statistical():
    '''
игровой агент стратегия которого заключается в том чтобы \
найти среди статистики наиболее часто встречающееся значение опонента, \
и вернуть противоположное по значимости значение
    '''

    def __init__(self):
        self.action_histogram = {}

    def __call__(self, observation, configuration):
        if observation.step == 0:
            # в случае начального раунда возвращаем 
            # инициализируем словарь исторических данных
            self.action_histogram = {}
            # возвращаем случаное значение
            return random.randrange(0, configuration.signs)
        # берем последнее значение оппонента и инкрементируем его счетчик
        action = observation.lastOpponentAction
        if action not in self.action_histogram:
            self.action_histogram[action] = 0
        self.action_histogram[action] += 1
        # находим наиболее часто встречающеемя значение
        mode_action = None
        mode_action_count = None
        for k, v in self.action_histogram.items():
            if mode_action_count is None or v > mode_action_count:
                mode_action = k
                mode_action_count = v
                continue
        # возвращаем его противозначение
        return  (mode_action + 1) % configuration.signs

agent = statistical()

# метод вызываемый из kaggle_environments
def call_agent(obs, conf):
    return agent(obs, conf)


Overwriting statistical_agent.py


In [37]:
# создаем уникальные комбинации агентов для тестирования
agent_comb = list(itertools.combinations(cmn.AGENTS.keys(), 2))
# определяем словарь результатов
results ={}
# запускаем тестирование агентов
for a1,  a2 in agent_comb:
    # получаем названия файлов тестируемых агентов
    af1 = cmn.AGENTS[a1]
    af2 = cmn.AGENTS[a2]
    # выполняем тестирование и сохраняем результаты
    results[(a1, a2)] = evaluate(
        "rps", 
        [af1, af2], 
        configuration={"episodeSteps": 100} )[0]
#print("\n"*3, "{\n" + "\n".join("   {!r}: {!r},".format(k, v) for k, v in results.items()).strip(',') + "\n}", "\n"*3)
# определяем победителей
winners = [k[0] if v[0] > v[1] else k[1] if v[0] < v[1] else None for k, v in results.items()]
# формируем и сортируем статистику
stats  = { t[0]:t[1] for t in sorted([(k, winners.count(k)) for k in cmn.AGENTS.keys()], key=lambda t: t[1], reverse=True)}
# выводим результаты
print("\n" + "\n".join("{!r}: {!r},".format(k, v) for k, v in stats.items()).strip(','), "\n")
print(f"Total agents count: {len(cmn.AGENTS)}\n")



'counter_reactionar': 7,
'spline_interp': 7,
'reactionary': 6,
'linear_interp': 6,
'statistical': 5,
'contr_opponent': 5,
'copy_opponent': 3,
'LinearRegression': 3,
'paper': 1,
'rock': 1,
'scissors': 1,
'loop': 1,
'randomizer': 0 

Total agents count: 13

